In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import time
import math
import socket

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import cma
from deap import creator
from deap import tools

import torch 
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad

import sys
sys.path.append("/Users/yashima/HKlab/modules_HKlab/")
import sim2d2link as sim
import helper as hp

device :  cuda
num gpu :  1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device('cuda') 
print('device : ', device)
print('num gpu : ', torch.cuda.device_count())

device :  cuda
num gpu :  1


In [56]:
model_fk = hp.BasicRegressor2()
model_fk.layer1 = nn.Linear(5, 2048)
model_fk.last = nn.Linear(2048, 3)
model_fk = nn.DataParallel(model_fk).to(device).eval().float()
model_fk.load_state_dict(torch.load("/Users/yashima/HKlab/Robotics/models/model_fk.pth"))

<All keys matched successfully>

In [57]:
def fk(init_theta1, init_theta2, init_theta3, init_theta4, init_theta5,
           a1, a2, a3, a4, a5):
    additional_cost = 0
    next_theta1 = np.clip(init_theta1 + a1, -180, 180)
    next_theta2 = np.clip(init_theta2 + a2, 0, 120)
    next_theta3 = np.clip(init_theta3 + a3, -180, 0)
    next_theta4 = np.clip(init_theta4 + a4, -180, 180)
    next_theta5 = np.clip(init_theta5 + a5, -160, 60)
    additional_cost += 1e2 if ((init_theta1+a1<-180)or(init_theta1+a1>180)) else 0
    additional_cost += 1e2 if ((init_theta2+a2<0)or(init_theta2+a2>120)) else 0
    additional_cost += 1e2 if ((init_theta3+a3<-180)or(init_theta3+a3>0)) else 0
    additional_cost += 1e2 if ((init_theta4+a4<-180)or(init_theta4+a4>180)) else 0
    additional_cost += 1e2 if ((init_theta5+a5<-160)or(init_theta5+a5>60)) else 0
    
    
    out = model_fk(torch.tensor([[next_theta1, next_theta2, next_theta3, next_theta4, next_theta5]]).float()/360)
    pos_pred = out.cpu().detach().numpy()[0]
    return next_theta1, next_theta2, next_theta3, next_theta4, next_theta5, pos_pred, additional_cost

# def evaluate(individual):
#     regulation_penalty = 0
#     a01, a02, a03, a04, a05, a11, a12, a13, a14, a15 = individual
#     theta11, theta12, theta13, theta14, theta15, pos_pred, additional_cost = fk(
#         INIT_THETA1, INIT_THETA2, INIT_THETA3, INIT_THETA4, INIT_THETA5,
#         a01, a02, a03, a04, a05)
#     cost1 = np.linalg.norm(pos_pred-POS_GOAL1)
#     regulation_penalty += additional_cost
#     theta21, theta22, theta23, theta24, theta25, pos_pred, additional_cost = fk(
#         theta11, theta12, theta13, theta14, theta15,
#         a11, a12, a13, a14, a15)
#     cost2 = np.linalg.norm(pos_pred-POS_GOAL2)
#     regulation_penalty += additional_cost    
#     return cost1+cost2+regulation_penalty, 

# def evaluate(individual):
#     regulation_penalty = 0
#     a01, a02, a03, a04, a05 = individual
#     theta11, theta12, theta13, theta14, theta15, pos_pred, additional_cost = fk(
#         INIT_THETA1, INIT_THETA2, INIT_THETA3, INIT_THETA4, INIT_THETA5,
#         a01, a02, a03, a04, a05)
#     cost1 = np.linalg.norm(pos_pred-POS_GOAL1)
#     regulation_penalty += additional_cost    
#     return cost1+regulation_penalty,

def evaluate(individual, init_theta, pos_goal):
    regulation_penalty = 0
    a01, a02, a03, a04, a05 = individual
    theta11, theta12, theta13, theta14, theta15, pos_pred, additional_cost = fk(
        init_theta[0], init_theta[1], init_theta[2], init_theta[3], init_theta[4], 
        a01, a02, a03, a04, a05)
    cost1 = np.linalg.norm(pos_pred-pos_goal)
    regulation_penalty += additional_cost
    
    return cost1+regulation_penalty,

In [58]:
def solve(IND_DIM, NPOP, NGEN, init_theta, pos_goal):
    t1 = time.time() 
    np.random.seed(64)

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
    strategy = cma.Strategy(centroid=[0.0]*IND_DIM, sigma=1.0, lambda_=NPOP)  # cnetroid: "mu", sigma: "sigma"

    toolbox = base.Toolbox()
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)
    toolbox.register("evaluate", evaluate, init_theta=init_theta,pos_goal=pos_goal)
    
    
    
    halloffame = tools.HallOfFame(1, similar=np.array_equal)

    mean_fitness = []
    halloffame_array = []
    C_array = []
    centroid_array = []
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals"] + (stats.fields if stats else [])
    fitness_history = []

    for gen in range(NGEN):
        stack_fitness = []
        population = toolbox.generate()
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            stack_fitness.append(fit)         
        toolbox.update(population)
        halloffame.update(population)

        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)

    t2 = time.time() 
    elapsed_time = t2-t1
    print(f"solving time: {elapsed_time:.3f}s")
    
    id_best = 0
    eval_best = 1e10
    for i in range(len(population)):
        eval = sum(toolbox.evaluate(population[i]))
        if eval < eval_best:
            id_best = i
            eval_best = eval
            
    
    
    return population[id_best]

In [59]:
def solve_sequence(solver_params, init_theta, pos_goals):
    all_commands = []
    for i in range(len(pos_goals)):
        conditions = {"init_theta":init_theta, "pos_goal":pos_goals[i]}
        solver_params.update(conditions)
        command = solve(**solver_params)
        init_theta = list(np.array(init_theta)+np.array(command))
        all_commands += list(command)
        if i!=0: all_commands[5*i:5*(i+1)] = list(np.array(all_commands[5*(i-1):5*i]) + np.array(all_commands[5*i:5*(i+1)]))
    return all_commands

In [68]:
solver_params = {"IND_DIM":5, "NPOP":6, "NGEN":120}
init_theta = [0., 0., 0., 0., 0.]
pos_goals = [np.array([[5, 5, 1]]), np.array([[3, 3, 3]])]

commands = solve_sequence(solver_params, init_theta, pos_goals)
print(commands)

C:\Users\yashima\Anaconda3\envs\torch-gpu\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\yashima\Anaconda3\envs\torch-gpu\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	min         	max           
0  	6     	[5.30706709]	[105.30923409]
1  	6     	[5.1524271] 	[205.28615226]
2  	6     	[5.05204185]	[105.02097272]
3  	6     	[5.01678636]	[105.14023244]
4  	6     	[5.08648676]	[105.13733677]
5  	6     	[4.91337373]	[105.21660179]
6  	6     	[4.72220595]	[5.17116452]  
7  	6     	[4.5487223] 	[104.52984175]
8  	6     	[4.41612722]	[4.85704126]  
9  	6     	[3.84501389]	[4.87269547]  
10 	6     	[3.46084442]	[104.42818224]
11 	6     	[2.47632102]	[103.60676609]
12 	6     	[2.0106941] 	[103.04161723]
13 	6     	[1.86490829]	[102.69145493]
14 	6     	[1.86190408]	[3.12328929]  
15 	6     	[0.88805388]	[2.38331998]  
16 	6     	[1.17744371]	[2.83161506]  
17 	6     	[1.19302656]	[103.22043082]
18 	6     	[1.29293417]	[102.21473297]
19 	6     	[1.19394159]	[2.88022431]  
20 	6     	[1.02540036]	[2.60911535]  
21 	6     	[1.38082463]	[3.96684434]  
22 	6     	[1.48573336]	[2.83594132]  
23 	6     	[1.03673703]	[1.60203476]  
24 	6     	[0.83879587]	[

97 	6     	[0.00037337]	[0.00083784]
98 	6     	[0.00038358]	[0.00088348]
99 	6     	[8.31041808e-05]	[0.00046369]
100	6     	[0.00010821]    	[0.00055583]
101	6     	[0.0002338]     	[0.00070061]
102	6     	[9.10865068e-05]	[0.000494]  
103	6     	[0.00011998]    	[0.00042662]
104	6     	[9.16675255e-05]	[0.00035846]
105	6     	[6.07041046e-05]	[0.00034246]
106	6     	[5.19402412e-05]	[0.00028057]
107	6     	[6.32456682e-05]	[0.00022546]
108	6     	[0.00011448]    	[0.00033668]
109	6     	[0.00010247]    	[0.00027816]
110	6     	[2.72319526e-05]	[0.00033404]
111	6     	[9.63995633e-05]	[0.00016585]
112	6     	[7.17889388e-05]	[0.00022638]
113	6     	[6.16693517e-05]	[0.00022243]
114	6     	[7.99914763e-05]	[0.00021201]
115	6     	[3.30646257e-05]	[7.73671044e-05]
116	6     	[3.50175132e-05]	[0.0001597]     
117	6     	[3.31984495e-05]	[0.00012193]    
118	6     	[4.75775013e-05]	[0.00010964]    
119	6     	[2.61902565e-05]	[8.79247508e-05]
solving time: 2.820s
[38.76348589528121, 45.4

In [61]:
def input_sequence(commands, length=10, interval=0):
    input_angles = []
    steps = int(len(commands)/5)
    for i in range(steps):
        if i!=0: input_angles += [commands[5*(i-1)],commands[5*(i-1)+1],commands[5*(i-1)+2],commands[5*(i-1)+3],commands[5*(i-1)+4]]*interval
        for j in range(length):
            for k in range(i*5, (i+1)*5):
                start = 0 if i==0 else commands[k-5]
                input_angles.append(np.linspace(start, commands[k], length)[j])

    return input_angles

In [66]:
# commands = [30.,30.,-30.,30.,30.]
# model_fk(torch.tensor([commands])/360)

tensor([[2.9282, 1.8237, 0.5625]], device='cuda:0', grad_fn=<AddmmBackward>)

In [69]:
HOST = '127.0.0.1'
PORT = 50008
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

command_sequense = input_sequence(commands, length=50, interval=10)
for i, command in enumerate(command_sequense):
    signal = str(command) if i==0 else signal+","+str(command)
    
client.sendto(signal.encode('utf-8'),(HOST,PORT))

10124